In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import pickle
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

from utils import extract_features_many_filenames

# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split

/Users/robertmoss/Code/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
# To save as normal python script (easier to git diff)
!jupyter nbconvert --to script classifier.ipynb

[NbConvertApp] Converting notebook classifier.ipynb to script
[NbConvertApp] Writing 3748 bytes to classifier.py


In [3]:
training_folders = {
    'vehicles': [
        'KITTI_extracted',
        'GTI_Right',
        'GTI_MiddleClose',
        'GTI_Left',
        'GTI_Far'
    ],
    'non-vehicles': [
        'GTI',
        'Extras'
    ]
}

training_filenames = {
    'vehicles': [],
    'non-vehicles': []
}

for folder, subfolders in training_folders.items():
    for subfolder in subfolders:
        files = glob.glob(folder + '/' + subfolder + '/' + '*.png')
        training_filenames[folder].extend(files)

In [4]:
# Prepare training data
    # Load image
    # Extract features from image
    # Add features to training set
    # Preprocess data (might not be here?)

In [5]:
# Smaller sample size while preparing pipeline
# sample_size = 500
# training_filenames['vehicles'] = training_filenames['vehicles'][0:sample_size]
# training_filenames['non-vehicles'] = training_filenames['non-vehicles'][0:sample_size]

In [6]:
# PARAMETERS
config = {
    'color_space': 'YUV',      # Can be RGB, HSV, LUV, HLS, YUV, YCrCb

    'spatial_feat': True,      # Spatial features on or off
    'hist_feat': True,         # Histogram features on or off
    'hog_feat': True,          # HOG features on or off
    
    # HOG features
    'orient': 11,               # HOG orientations
    'pix_per_cell': 8,         # HOG pixels per cell
    'cell_per_block': 2,       # HOG cells per block
    'hog_channel': 'ALL',      # Can be 0, 1, 2, or "ALL"
    
    # Spatial features
    'spatial_size': (16, 16),  # Spatial binning dimensions

    # Color hist features
    'hist_bins': 32,           # Number of histogram bins (for color histogram feature)
    'hist_range': (0, 256)     # Range for color histogram
}

In [7]:
%%time
# Feature extraction and preparation
car_features = extract_features_many_filenames(training_filenames['vehicles'], **config)
notcar_features = extract_features_many_filenames(training_filenames['non-vehicles'], **config)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2)

print('Feature vector length:', len(X_train[0]))

/Users/robertmoss/Code/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Feature vector length: 7332
CPU times: user 3min, sys: 20.1 s, total: 3min 20s
Wall time: 6min 17s


In [8]:
%%time
# Create the classifier
svc = LinearSVC()

# Train the classifier
svc.fit(X_train, y_train)

# Classifier accuracy
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

Test Accuracy of SVC =  0.9938
CPU times: user 20.2 s, sys: 5.59 s, total: 25.7 s
Wall time: 57.1 s


In [10]:
# Save the classifier (and the config for training so feature preparation
# is the same for training and prediction when classifier is loaded)
pickle.dump(svc, open('svc_classifier.pkl', 'wb'))
pickle.dump(config, open('feature_config.pkl', 'wb'))
pickle.dump(X_scaler, open('x_scaler.pkl', 'wb'))